## The [Architecture of Complexity] by Herbert A. Simon

There once were two watchmakers, named Hora and Tempus, who manufactured very
fine watches. Both of them were highly regarded, and the phones in their
workshops rang frequently -new customers were constantly calling them. However,
Hora prospered, while Tempus became poorer and poorer and finally lost his shop.
What was the reason?

The watches the men made consisted of about 1,000 parts each. Tempus had so
constructed his that if he had one partly assembled and had to put it down - to
answer the phone say - it immediately fell to pieces and had to be reassembled
from the elements. The better the customers liked his watches, the more they
phoned him, the more difficult it became for him to find enough uninterrupted
time to finish a watch.

The watches that Hora made were no less complex than those of Tempus. But he had
designed them so that he could put together subassemblies of about ten elements
each. Ten of these subassemblies, again, could be put together into a larger
subassembly; and a system of ten of the latter subassemblies constituted the
whole watch. Hence, when Hora had to put down a partly assembled watch in order
to answer the phone, he lost only a small part of his work, and he assembled his
watches in only a fraction of the man-hours it took Tempus. 

[Architecture of Complexity]: https://www2.econ.iastate.edu/tesfatsi/ArchitectureOfComplexity.HSimon1962.pdf

In [1]:
import sys
sys.path.append("../")

from tools import (
    web_search,
    search,
    page_scraper,
    pick
)
import json

In [2]:
import asyncio


async def test_wait(timeout=2, seconds_passed=0):
    await asyncio.sleep(timeout)
    seconds_passed += timeout
    return "done"

sequence = ["one", "two", "three"]


async def test(sequence=sequence):
    seconds_passed = 0
    dones = []
    for step in sequence:
        print(f"Starting {step}")
        result = await test_wait(1, seconds_passed)
        print("\n")
        print(f"Finished {step}")
        dones.append(result)
    return dones

# response = await test()
# print(response)

In [13]:
import asyncio
import re

from functools import partial, wraps

from intent import catalog, get_intent

def async_wrap(func):
    @wraps(func)
    async def run(*args, loop=None, executor=None, **kwargs):
        if loop is None:
            loop = asyncio.get_event_loop()
        pfunc = partial(func, *args, **kwargs)
        return await loop.run_in_executor(executor, pfunc)
    return run

async def spool(
    history,
    max_links=3,
    link=0,
    previous_intents=[]
):
    """

    Spins up an interactive and asynchronous communication channel with the
    following services:

    - Intent Processor
    - Function Payloader
    - Actions (Functions to Call)
    - IsDone (Determines if the conversation is done)

    """
    # print(f"Link # {link}")
    if link >= max_links:
        return history
    else:
        try:
            # get the first user message
            first_user_msg = next(
                (h["content"] for h in history if h["role"] == "user"),
                None
            )

            last_user_msg = next(
                (h["content"]
                 for h in reversed(history) if h["role"] == "user"),
                None
            )

            last_asst_msg = next(
                (h["content"]
                 for h in reversed(history) if h["role"] == "assistant"),
                None
            )

            if last_asst_msg:
                padding = re.compile(r"\n{2,}")
                summary = padding.sub("\n\n", last_asst_msg)[:200]
                print(f"Summary: {summary}")
            
            print(f"\nInstruction: {last_user_msg}\n")
            
            # get the intent for the message
            intent = get_intent(last_user_msg)

            fn, fu, id = intent["function"], intent["follow_up"], intent["name"]
            # convert func to async so we can await it
            func = async_wrap(fn)

            if id in previous_intents:
                print(f"🔥 Intent drained: {id}")
                return history
            else:
                previous_intents.append(id)
            print(f"Intent: 🔍 `{id}`")

            # This needs to be handled by the payloader in the func(history)
            if id == "elaborate_vtx_url":
                call = await func(last_asst_msg)
            elif id == "llm_pick":
                call = await func(first_user_msg, last_asst_msg)
            elif id == "llm_eloquent":
                call = await func(f"Query: {first_user_msg}\n Content:{last_asst_msg}")
            else:
                call = await func(last_user_msg)

            history.append({
                "role": "assistant",
                "content": json.dumps(call) if type(call) != str else call
            })

            if fu:
                history.append({
                    "role": "user",
                    "content": fu
                })
                return await spool(
                    history,
                    max_links,
                    link + 1,
                    previous_intents
                )
            else:
                print("--- DONE ---")
                return history
        except Exception as e:
            print(f"🔥 Error: {e}")
            return history

In [14]:
test_history = [
    # {"role": "user", "content": "What products does Vertex have?"},
    {"role": "user", "content": "How do I contact Vertex Inc customer service?"},
]

In [15]:
result = await spool(test_history, 5)


Instruction: How do I contact Vertex Inc customer service?

Intent: 🔍 `search_vtx_com`
Summary: [{"title": "Contact Us for Support | Vertex Inc.", "url": "https://www.vertexinc.com/contact-us", "snippet": "For Vertex Cloud support: **Phone: 1-855-221-5885** Email: cloudsupport@vertexinc.com"}, {

Instruction: The results are a list of pages. I will pick the most relevant one for you.

Intent: 🔍 `llm_pick`
Summary: The best url to contact Vertex Inc customer service is: 
https://www.vertexinc.com/contact-us

Instruction: The selected URL needs additional context. I will elaborate on it.

Intent: 🔍 `elaborate_vtx_url`
Summary: Contact Us for Support | Vertex Inc.

Skip To Primary Navigation
Skip To Main Content
Skip To Footer

Menu

1. Vertex Exchange

* Search

Search

Search

Log In

* Vertex Cloud Login
* Vertex Communit

Instruction: Text is verbose, I will summarize it for you.

Intent: 🔍 `llm_eloquent`
--- DONE ---


In [16]:
#print(json.dumps(result, indent=2))

In [17]:
import re

from IPython.display import Markdown, display

final_response = result[-1]["content"]
display(Markdown(final_response))

- To contact Vertex Inc customer service, you can visit their website and go to the "Contact Us" page.
- For product support, you can visit the Vertex Community (myVertex) for self-help support or call 1-800-281-1900 (North America) or +1-610-640-4210 (International).
- For Vertex Cloud support, you can call 1-855-221-5885 or email cloudsupport@vertexinc.com.
- For training inquiries, you can visit the Training page on their website or call 1-877-455-2645 (North America) or +1–484–595–2590 (International).
